**Set environment**

In [1]:
source ../config_duke.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



## Set region
`GATA1 chr X: 47786400 -  49786750`

`MYC   chr 8:126735900 - 128736250`

`FADS  chr11: 61787325 -  61898350`

**Set region by base**

In [2]:
START=47786400
END=49786750
CHROM=chrX

FP_OUT=${FD_RES}/Tewhey_K562_TileMPRA/coverage/region_GATA1_perbase.bed
cat /dev/null > ${FP_OUT}
for LOC in $(seq ${START} ${END}); do
    echo -e "${CHROM}\\t${LOC}\\t$((${LOC} + 1))" >> ${FP_OUT}
done

In [3]:
START=126735900
END=128736250
CHROM=chr8

FP_OUT=${FD_RES}/Tewhey_K562_TileMPRA/coverage/region_MYC_perbase.bed
cat /dev/null > ${FP_OUT}
for LOC in $(seq ${START} ${END}); do
    echo -e "${CHROM}\\t${LOC}\\t$((${LOC} + 1))" >> ${FP_OUT}
done

In [4]:
START=61787325
END=61898350
CHROM=chr11

FP_OUT=${FD_RES}/Tewhey_K562_TileMPRA/coverage/region_FADS_perbase.bed
cat /dev/null > ${FP_OUT}
for LOC in $(seq ${START} ${END}); do
    echo -e "${CHROM}\\t${LOC}\\t$((${LOC} + 1))" >> ${FP_OUT}
done

In [5]:
head -3 ${FD_RES}/Tewhey_K562_TileMPRA/coverage/region*perbase.bed

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/region_FADS_perbase.bed <==
chr11	61787325	61787326
chr11	61787326	61787327
chr11	61787327	61787328

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/region_GATA1_perbase.bed <==
chrX	47786400	47786401
chrX	47786401	47786402
chrX	47786402	47786403

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/region_MYC_perbase.bed <==
chr8	126735900	126735901
chr8	126735901	126735902
chr8	126735902	126735903


In [6]:
tail -n 3 ${FD_RES}/Tewhey_K562_TileMPRA/coverage/region*perbase.bed

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/region_FADS_perbase.bed <==
chr11	61898348	61898349
chr11	61898349	61898350
chr11	61898350	61898351

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/region_GATA1_perbase.bed <==
chrX	49786748	49786749
chrX	49786749	49786750
chrX	49786750	49786751

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/region_MYC_perbase.bed <==
chr8	128736248	128736249
chr8	128736249	128736250
chr8	128736250	128736251


## Calculate coverage perbase for input and output

**Test**

In [7]:
### global variables
FD_CNT=${FD_RES}/Tewhey_K562_TileMPRA/count
FD_COV=${FD_RES}/Tewhey_K562_TileMPRA/coverage

IDX_COLS=(8 9)
IDN_COLS=(Input Output)
IDX=1

### loop through each region
REGIONS=(GATA1 MYC FADS)
for REGION in ${REGIONS[@]}; do

echo "Region: " ${REGION}
IDX_COL=${IDX_COLS[${IDX}]}
IDN_COL=${IDN_COLS[${IDX}]}

FP_CNTS=($(ls ${FD_CNT}/*${REGION}*.round.bed))
for FP_INP in ${FP_CNTS[@]}; do
    FN_INP=$(basename ${FP_INP})
    FN_OUT=${FN_INP%.*.*}.${IDN_COL}.perbase.tsv.gz
    echo "FN_INP:" ${FN_INP}
    echo "FN_OUT:" ${FN_OUT}
done
echo

done

Region:  GATA1
FN_INP: Tile_K562_hg38_20210130.GATA1.stranded_pos.round.bed
FN_OUT: Tile_K562_hg38_20210130.GATA1.stranded_pos.Output.perbase.tsv.gz

Region:  MYC
FN_INP: Tile_K562_hg38_20210130.MYC.stranded_pos.round.bed
FN_OUT: Tile_K562_hg38_20210130.MYC.stranded_pos.Output.perbase.tsv.gz

Region:  FADS
FN_INP: Tile_K562_hg38_20200905.FADS.stranded_neg.round.bed
FN_OUT: Tile_K562_hg38_20200905.FADS.stranded_neg.Output.perbase.tsv.gz
FN_INP: Tile_K562_hg38_20200905.FADS.stranded_pos.round.bed
FN_OUT: Tile_K562_hg38_20200905.FADS.stranded_pos.Output.perbase.tsv.gz



**Run**

In [8]:
### loop through each region
REGIONS=(GATA1 MYC FADS)
for REGION in ${REGIONS[@]}; do

sbatch -p ${NODE} \
    --mem 8G \
    --array 0-1 \
    --export=REGION=${REGION} \
    -o ${FD_LOG}/TMPRA_coverage_${REGION}.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
source ../config_load_module_bedtools.sh

FD_CNT=${FD_RES}/Tewhey_K562_TileMPRA/count
FD_COV=${FD_RES}/Tewhey_K562_TileMPRA/coverage

IDX_COLS=(8 9)
IDN_COLS=(Input Output)

### set input and output
IDX_COL=${IDX_COLS[${SLURM_ARRAY_TASK_ID}]}
IDN_COL=${IDN_COLS[${SLURM_ARRAY_TASK_ID}]}

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through files
FP_CNTS=($(ls ${FD_CNT}/*${REGION}*.round.bed))
for FP_INP in ${FP_CNTS[@]}; do
    FN_INP=$(basename ${FP_INP})
    FN_OUT=${FN_INP%.*.*}.${IDN_COL}.perbase.tsv.gz
    FP_OUT=${FD_COV}/${FN_OUT}
    echo ++++++++++++++++++++++++++++++++++++++
    echo
    
    ### show input file
    echo "Input: " ${FP_INP}
    echo
    echo "show first few lines of input"
    cat ${FP_INP} | head -5
    echo
    echo "show last few lines of input"
    cat ${FP_INP} | tail -n 5
    echo
    
    ### execute
    bedtools map \
        -a ${FD_COV}/region_${REGION}_perbase.bed \
        -b ${FP_INP} \
        -o mean \
        -c ${IDX_COL} |\
        gzip -c > ${FP_OUT}
    
    ### show output file
    echo "Output: " ${FP_OUT}
    echo
    echo "show first few lines of output:"
    zcat ${FP_OUT} | head -5
    echo
    echo "show last few lines of output"
    zcat ${FP_OUT} | tail -n 5
    echo
    
done
echo

EOF

done

Submitted batch job 28275607
Submitted batch job 28275608
Submitted batch job 28275609


**Check log**

In [12]:
cat ${FD_LOG}/TMPRA_coverage_GATA1.1.txt

Slurm Array Index:  1
Time Stamp:         05-17-22+16:26:31

++++++++++++++++++++++++++++++++++++++

Input:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.GATA1.stranded_pos.round.bed

show first few lines of input
chrX	47786401	47786600	X:47786401-47786600	.	+	2.306	2117.787	10474.869
chrX	47786451	47786650	X:47786451-47786650	.	+	0.783	1386.007	2387.119
chrX	47786501	47786700	X:47786501-47786700	.	+	-0.129	1955.626	1788.236
chrX	47786551	47786750	X:47786551-47786750	.	+	0.588	2388.997	3591.264
chrX	47786601	47786800	X:47786601-47786800	.	+	0.067	2102.916	2202.385

show last few lines of input
chrX	49786351	49786550	X:49786351-49786550	.	+	-0.216	775.951	668.187
chrX	49786401	49786600	X:49786401-49786600	.	+	0.18	1556.617	1763.467
chrX	49786451	49786650	X:49786451-49786650	.	+	-0.373	127.91	98.961
chrX	49786501	49786700	X:49786501-49786700	.	+	-0.49	120.701	86.235
chrX	49786551	49786750	X:49786551-49786750	.	+	-0.272	728

In [12]:
cat ${FD_LOG}/TMPRA_coverage_FADS.0.txt

Slurm Array Index:  0
Time Stamp:         05-17-22+17:09:02

++++++++++++++++++++++++++++++++++++++

Input:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20200905.FADS.stranded_neg.round.bed

show first few lines of input
chr11	61787329	61787528	11:61554801-61555000_RC	.	-	-0.882	325.845	176.742
chr11	61787334	61787533	11:61554806-61555005_RC	.	-	-0.832	336.927	189.098
chr11	61787339	61787538	11:61554811-61555010_RC	.	-	-0.391	310.188	236.291
chr11	61787344	61787543	11:61554816-61555015_RC	.	-	-0.768	580.396	341.076
chr11	61787349	61787548	11:61554821-61555020_RC	.	-	-1.093	544.991	255.805

show last few lines of input
chr11	61898129	61898328	11:61665601-61665800_RC	.	-	2.639	153.418	956.769
chr11	61898134	61898333	11:61665606-61665805_RC	.	-	1.782	215.673	742.172
chr11	61898139	61898338	11:61665611-61665810_RC	.	-	1.959	269.52	1048.729
chr11	61898144	61898343	11:61665616-61665815_RC	.	-	3.217	199.813	1859.036
chr11	61898149	6189

-----

## GATA1

In [25]:
sbatch -p ${NODE} \
    --mem 8G \
    --array 0-1 \
    -o ${FD_LOG}/TMPRA_coverage_gata1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
source ../config_load_module_bedtools.sh

REGION="GATA1"

FD_BED=${FD_RES}/Tewhey_K562_TileMPRA/count
FD_COV=${FD_RES}/Tewhey_K562_TileMPRA/coverage

IDX_COLS=(8 9)
IDN_COLS=(Input Output)

### set input and output
IDX_COL=${IDX_COLS[${SLURM_ARRAY_TASK_ID}]}
IDN_COL=${IDN_COLS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=Tile_K562_hg38_20210130.GATA1.count.bed
FN_OUT=Tile_K562_hg38_20210130.GATA1.${IDN_COL}.perbase.tsv.gz

FP_INP=${FD_BED}/${FN_INP}
FP_OUT=${FD_COV}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo "Input:             " ${FP_INP}
echo "Output:            " ${FP_OUT}
echo

### show first few lines of input
echo ${FP_INP}
cat ${FP_INP} | head -n 5
echo

### show last few lines of input
echo ${FP_INP}
cat ${FP_INP} | tail -n 5
echo

### execute
bedtools map \
    -a ${FD_COV}/region_${REGION}.bed \
    -b ${FP_INP} \
    -o mean \
    -c ${IDX_COL} |\
    gzip -c > ${FP_OUT}


### show first few lines of output
echo ${FP_OUT}
zcat ${FP_OUT} | head -n 5
echo

### show first few lines of output
echo ${FP_OUT}
zcat ${FP_OUT} | tail -n 5
echo


### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28235762


In [27]:
cat ${FD_LOG}/TMPRA_coverage_gata1.0.txt

Slurm Array Index:  0
Time Stamp:         05-11-22+17:02:50
Input:              /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.GATA1.count.bed
Output:             /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/Tile_K562_hg38_20210130.GATA1.Input.perbase.tsv.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.GATA1.count.bed
chrX	47786401	47786600	X:47786401-47786600	.	+	2.306	2117.787	10474.869
chrX	47786451	47786650	X:47786451-47786650	.	+	0.783	1386.007	2387.119
chrX	47786501	47786700	X:47786501-47786700	.	+	-0.129	1955.626	1788.236
chrX	47786551	47786750	X:47786551-47786750	.	+	0.588	2388.997	3591.264
chrX	47786601	47786800	X:47786601-47786800	.	+	0.067	2102.916	2202.385

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.GATA1.count.bed
chrX	49786351	49786

In [26]:
ls ${FD_RES}/Tewhey_K562_TileMPRA/coverage

region_GATA1.bed          Tile_K562_20210130.GATA1.unstranded.bin1000.bed.gz
region_GATA1_bin1000.bed  Tile_K562_20210130.GATA1.unstranded.bin100.bed.gz
region_GATA1_bin100.bed   Tile_K562_20210130.GATA1.unstranded.bin1.bed.gz
region_GATA1_bin1.bed     Tile_K562_20210130.GATA1.unstranded.bin200.bed.gz
region_GATA1_bin200.bed   Tile_K562_20210130.GATA1.unstranded.bin500.bed.gz
region_GATA1_bin500.bed   Tile_K562_hg38_20210130.GATA1.Input.perbase.tsv.gz
region_MYC.bed            Tile_K562_hg38_20210130.GATA1.Output.perbase.tsv.gz


In [28]:
zcat ${FD_RES}/Tewhey_K562_TileMPRA/coverage/Tile_K562_hg38_20210130.GATA1.Input.perbase.tsv.gz | head

chrX	47786400	47786401	.
chrX	47786401	47786402	2117.787
chrX	47786402	47786403	2117.787
chrX	47786403	47786404	2117.787
chrX	47786404	47786405	2117.787
chrX	47786405	47786406	2117.787
chrX	47786406	47786407	2117.787
chrX	47786407	47786408	2117.787
chrX	47786408	47786409	2117.787
chrX	47786409	47786410	2117.787

gzip: stdout: Broken pipe


## MYC

In [3]:
sbatch -p ${NODE} \
    --mem 8G \
    --array 0-1 \
    -o ${FD_LOG}/TMPRA_coverage_myc.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh

REGION="MYC"

FD_BED=${FD_RES}/Tewhey_K562_TileMPRA/count
FD_COV=${FD_RES}/Tewhey_K562_TileMPRA/coverage

IDX_COLS=(8 9)
IDN_COLS=(Input Output)

### set input and output
IDX_COL=${IDX_COLS[${SLURM_ARRAY_TASK_ID}]}
IDN_COL=${IDN_COLS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=Tile_K562_hg38_20210130.${REGION}.count.bed
FN_OUT=Tile_K562_hg38_20210130.${REGION}.${IDN_COL}.perbase.tsv.gz

FP_INP=${FD_BED}/${FN_INP}
FP_OUT=${FD_COV}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo "Input:             " ${FP_INP}
echo "Output:            " ${FP_OUT}
echo

### show first few lines of input
echo ${FP_INP}
cat ${FP_INP} | head -n 5
echo

### show last few lines of input
echo ${FP_INP}
cat ${FP_INP} | tail -n 5
echo

### execute
bedtools map \
    -a ${FD_COV}/region_${REGION}.bed \
    -b ${FP_INP} \
    -o mean \
    -c ${IDX_COL} |\
    gzip -c > ${FP_OUT}


### show first few lines of output
echo ${FP_OUT}
zcat ${FP_OUT} | head -n 5
echo

### show first few lines of output
echo ${FP_OUT}
zcat ${FP_OUT} | tail -n 5
echo


### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28236156


In [4]:
cat ${FD_LOG}/TMPRA_coverage_myc.0.txt

Slurm Array Index:  0
Time Stamp:         05-12-22+10:08:46
Input:              /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.MYC.count.bed
Output:             /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage/Tile_K562_hg38_20210130.MYC.Input.perbase.tsv.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.MYC.count.bed
chr8	126735901	126736100	8:126735901-126736100	.	+	3.216	889.87	8270.081
chr8	126735951	126736150	8:126735951-126736150	.	+	2.742	125.835	840.939
chr8	126736001	126736200	8:126736001-126736200	.	+	2.973	510.316	4005.357
chr8	126736051	126736250	8:126736051-126736250	.	+	0.518	702.549	1005.359
chr8	126736101	126736300	8:126736101-126736300	.	+	0.854	551.083	995.47

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/count/Tile_K562_hg38_20210130.MYC.count.bed
chr8	128735851	128